In [1]:
import sys
import os
from dotenv import load_dotenv

load_dotenv()
work_dir = os.getenv('WORK_DIR')
sys.path.append(work_dir)

In [2]:
from src.model.models import CardioTrainNormalize, GlucoseTypes, CholesterolTypes
from src.database.dbconnection import getconnection
from sqlalchemy import inspect
from sqlalchemy.orm import sessionmaker
from sqlalchemy.exc import SQLAlchemyError
from transform.TransformData import DataTransform

In [3]:
engine = getconnection()
Session = sessionmaker(bind=engine)
session = Session()

Conected successfully to database proyect!


In [4]:
try:
    if inspect(engine).has_table('GlucoseTypes'):
        if inspect(engine).has_table('CardioTrainNormalize'):
            CardioTrainNormalize.__table__.drop(engine)
        GlucoseTypes.__table__.drop(engine, checkfirst=True)
    GlucoseTypes.__table__.create(engine)
    print("Table created successfully.")
except SQLAlchemyError as e:
    print(f"Error creating table: {e}")
finally:
    engine.dispose()

Table created successfully.


In [5]:
try:
    if inspect(engine).has_table('CholesterolTypes'):
        if inspect(engine).has_table('CardioTrainNormalize'):
            CardioTrainNormalize.__table__.drop(engine)
        CholesterolTypes.__table__.drop(engine, checkfirst=True)
    CholesterolTypes.__table__.create(engine)
    print("Table created successfully.")
except SQLAlchemyError as e:
    print(f"Error creating table: {e}")
finally:
    engine.dispose()

Table created successfully.


In [6]:
try:
    if inspect(engine).has_table('CardioTrainNormalize'):
        CardioTrainNormalize.__table__.drop(engine)
    CardioTrainNormalize.__table__.create(engine)
    print("Table created successfully.")
except SQLAlchemyError as e:
    print(f"Error creating table: {e}")
finally:
    engine.dispose()

Table created successfully.


In [7]:
try:
    file = DataTransform('../data/cardio_train.csv')
    file.gender_by_category()
    file.cholesterol_by_category()
    file.gluc_by_category()
    file.IMC()
    file.days_to_age()


     
    transform_gluc = file.nomalize_gluc()
    transform_cholesterol = file.normalize_cholesterol()

    
    transform_gluc.to_sql('GlucoseTypes', con=engine, if_exists='append', index=False)
    transform_cholesterol.to_sql('CholesterolTypes', con=engine, if_exists='append', index=False)
    file.df.to_sql('CardioTrainNormalize', con=engine, if_exists='append', index=False)

    print("Data uploaded")

except SQLAlchemyError as e:
    print(f"Database error: {e}")

except Exception as e:
    print(f"Error: {e}")

finally:
    if hasattr(engine, 'dispose'):
        engine.dispose()
    if 'session' in locals():
        session.close()

Data uploaded
